In [2]:
import os
import numpy as np
import cv2
from keras.datasets import cifar10
import os
import numpy as np
import cv2
from keras.datasets import cifar10
from keras.utils import multi_gpu_model
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, AveragePooling2D, ZeroPadding2D
from keras.layers.core import Activation, Flatten
from keras.models import Sequential, load_model
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import time
os.chdir("C:/Users/wutti/face/")

Using TensorFlow backend.


In [3]:
img_size = 50
n_classes = 10
epch = 10
batch_size = 20


In [4]:
predict_person = {0:"unknown", 1:"Hung"}

x_train = []
y_train = []
x_test = []
y_test = []

i = 0
fold_train = os.listdir("train")
for cate_train in fold_train:
    img_train = os.listdir("train/"+cate_train)
    for numImg_train in img_train:
        try:
            name = 'train/'+cate_train+'/'+numImg_train
            img_array = cv2.imread(name)/255.0
            new_array = cv2.resize(img_array, (img_size, img_size))
            x_train.append(new_array)
            y_train.append(i)
        except Exception as e:
            pass
    i += 1

j = 0
fold_test = os.listdir("test")
for cate_test in fold_train:
    img_test = os.listdir("test/"+cate_test)
    for numImg_test in img_test:
        try:
            name = 'test/'+cate_test+'/'+numImg_test
            img_array = cv2.imread(name)/255.0
            new_array = cv2.resize(img_array, (img_size, img_size))
            x_test.append(new_array)
            y_test.append(j)
        except Exception as e:
            pass
    j += 1

X_train = np.asarray(x_train).astype('float32')
Y_train = np.asarray(y_train).astype('int')
X_test = np.asarray(x_test).astype('float32')
Y_test = np.asarray(y_test).astype('int')

print(X_train.shape)
print(Y_train.shape)
print(Y_train)
print(X_test.shape)
print(Y_test.shape)
print(Y_test)

(280, 50, 50, 3)
(280,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
(61, 50, 50, 3)
(61,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [39]:
model = Sequential()

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2))) 

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax'))
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

#train
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
callbacks_list = [checkpoint]

model.fit(X_train, Y_train, epochs=epch, batch_size=batch_size, 
            verbose=1, validation_data=(X_test, Y_test), callbacks=callbacks_list)

model.load_weights("weights.best.hdf5")
model.save('faceRecognize.model')
print("Finished")

Train on 280 samples, validate on 61 samples
Epoch 1/10
280/280 [==============================] - 3s 11ms/step - loss: 1.2526 - acc: 0.4607 - val_loss: 0.7421 - val_acc: 0.3770

Epoch 00001: val_acc improved from -inf to 0.37705, saving model to weights.best.hdf5
Epoch 2/10
280/280 [==============================] - 2s 7ms/step - loss: 0.7631 - acc: 0.6429 - val_loss: 0.5333 - val_acc: 0.7377

Epoch 00002: val_acc improved from 0.37705 to 0.73770, saving model to weights.best.hdf5
Epoch 3/10
280/280 [==============================] - 2s 9ms/step - loss: 0.3882 - acc: 0.8393 - val_loss: 0.1173 - val_acc: 0.9508

Epoch 00003: val_acc improved from 0.73770 to 0.95082, saving model to weights.best.hdf5
Epoch 4/10
280/280 [==============================] - 3s 9ms/step - loss: 0.0666 - acc: 0.9643 - val_loss: 0.0015 - val_acc: 1.0000

Epoch 00004: val_acc improved from 0.95082 to 1.00000, saving model to weights.best.hdf5
Epoch 5/10
280/280 [==============================] - 3s 9ms/step - l

In [4]:
new_model = load_model('faceRecognize.model')
predictions = new_model.predict(X_test)

scores = new_model.evaluate(X_test, Y_test, verbose=1)
print('Test accuracy:', scores[1])

print("Predicted", np.argmax(predictions, axis=1))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
61/61 [==============================] - 0s 2ms/step
Test accuracy: 1.0
Predicted [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [4]:
# save image

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
#eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
#mouth_cascade = cv2.CascadeClassifier('haarcascade_mcs_mouth.xml')
#smile_cascade = cv2.CascadeClassifier('haarcascade_smile.xml')
cap = cv2.VideoCapture(0)
i = 150
while(True):
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        cv2.imwrite("train/hung/face (%d).jpg" %(i+1), roi_color)
        time.sleep(0.3)
        i += 1
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        #eyes = eye_cascade.detectMultiScale(roi_gray)
        #for (ex,ey,ew,eh) in eyes:
        #    cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
            
    cv2.imshow('img',img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
# test video

predict_person = {0:"unknown", 1:"Hung"}
new_model = load_model('faceRecognize.model')
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)

font                   = cv2.FONT_HERSHEY_SIMPLEX
fontScale              = 1
fontColor              = (255,255,255)
lineType               = 2
try:
    while(True):
        ret, img = cap.read()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x,y,w,h) in faces:
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = img[y:y+h, x:x+w]

            temp1 = cv2.resize(roi_color,(img_size,img_size))/255.00
            temp2 = np.asarray(temp1).astype('float32')
            roi_face = temp2.reshape((1,img_size,img_size,3))
            predictions = new_model.predict(roi_face)
            result_predict = np.argmax(predictions, axis=1)
            
            cv2.putText(img, predict_person[result_predict[0]], (x+w,y+h), font, fontScale,fontColor,lineType)
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)

        cv2.imshow('img',img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
except Exception as e:
    cap.release()
    cv2.waitKey(0)
    cv2.destroyAllWindows()
cap.release()
cv2.waitKey(0)
cv2.destroyAllWindows()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [ ]:
cap.release()
cv2.waitKey(0)
cv2.destroyAllWindows()